In [2025]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2026]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters
import datetime



In [2029]:
# Extract timed event records

import pandas_gbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
    '/Users/veesheenyuen/Desktop/DataScience/Keys/saa-analytics-7c8937b70609.json',
    
    
)

sql1="""
SELECT NAME, RESULT, AGE, RANK AS COMPETITION_RANK, EVENT, DOB, COUNTRY, DIVISION, CATEGORY_EVENT, GENDER, COMPETITION, DATE
FROM `saa-analytics.results.saa_full` 
WHERE CATEGORY_EVENT='Throw' AND COMPETITION='NSG' AND RESULT!='NM' AND RESULT!='-' AND RESULT!='FOUL' AND RANK!='DNS' AND RESULT!='DNS' AND RESULT!='DNF' AND RESULT!='DNQ' AND RESULT!='DQ' AND RESULT IS NOT NULL
"""

athletes = pandas_gbq.read_gbq(sql1, project_id="saa-analytics", credentials=credentials)




Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|


In [2030]:
athletes

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,DIVISION,CATEGORY_EVENT,GENDER,COMPETITION,DATE
0,DANIEL WEE TECK JIN,35.6,None,6.0,Javelin (700g),None,None,A,Throw,Male,NSG,2023
1,RAJENDRAKUMAR DEESANTH,30.93,None,8.0,Javelin (700g),None,None,A,Throw,Male,NSG,2023
2,QUAH XIAO WEI,24.27,None,9.0,Javelin (700g),None,None,A,Throw,Male,NSG,2023
3,"CHENG BING QUAN, ARTHUR",50.38,None,1.0,Javelin (700g),None,None,A,Throw,Male,NSG,2023
4,ZHENG YIHONG,34.61,None,7.0,Javelin (700g),None,None,A,Throw,Male,NSG,2023
...,...,...,...,...,...,...,...,...,...,...,...,...
392,Leah Wong Li Wern,7.41,None,7.0,Shot Put (3.00kg),None,None,C,Throw,Female,NSG,2024
393,Nishka Sundar,6.71,None,11.0,Shot Put (3.00kg),None,None,C,Throw,Female,NSG,2024
394,Lin Hongyu,9.36,None,2.0,Shot Put (3.00kg),None,None,C,Throw,Female,NSG,2024
395,Long Yun Xin Naomi,6.83,None,10.0,Shot Put (3.00kg),None,None,C,Throw,Female,NSG,2024


In [2037]:
athletes_throwers=athletes[athletes['DIVISION']=='A']

In [2038]:
athletes_throwers

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,DIVISION,CATEGORY_EVENT,GENDER,COMPETITION,DATE
0,DANIEL WEE TECK JIN,35.6,None,6.0,Javelin (700g),None,None,A,Throw,Male,NSG,2023
1,RAJENDRAKUMAR DEESANTH,30.93,None,8.0,Javelin (700g),None,None,A,Throw,Male,NSG,2023
2,QUAH XIAO WEI,24.27,None,9.0,Javelin (700g),None,None,A,Throw,Male,NSG,2023
3,"CHENG BING QUAN, ARTHUR",50.38,None,1.0,Javelin (700g),None,None,A,Throw,Male,NSG,2023
4,ZHENG YIHONG,34.61,None,7.0,Javelin (700g),None,None,A,Throw,Male,NSG,2023
...,...,...,...,...,...,...,...,...,...,...,...,...
309,Emma Tan Yi Min,9.73,None,3.0,Shot Put (4.00kg),None,None,A,Throw,Female,NSG,2024
310,Foo Wen Qi,7.22,None,7.0,Shot Put (4.00kg),None,None,A,Throw,Female,NSG,2024
311,Isabelle Gan Jia Xin,7.93,None,5.0,Shot Put (4.00kg),None,None,A,Throw,Female,NSG,2024
312,Koh Wen-Rong,6.42,None,8.0,Shot Put (4.00kg),None,None,A,Throw,Female,NSG,2024


In [2039]:
athletes_throwers['RESULT'] = athletes_throwers['RESULT'].astype('float')

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/2969462197.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_throwers['RESULT'] = athletes_throwers['RESULT'].astype('float')


In [2040]:
athletes_throwers.sort_values(['GENDER', 'EVENT', 'RESULT'], ascending=[True, True, False], inplace=True)
athletes_throwers['overall_rank'] = 1
athletes_throwers['overall_rank'] = athletes_throwers.groupby(['EVENT', 'GENDER'])['overall_rank'].cumsum()

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/2190790224.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_throwers.sort_values(['GENDER', 'EVENT', 'RESULT'], ascending=[True, True, False], inplace=True)
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/2190790224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  athletes_throwers['overall_rank'] = 1
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_20211/2190790224.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [2041]:
athletes_throwers

,NAME,RESULT,AGE,COMPETITION_RANK,EVENT,DOB,COUNTRY,DIVISION,CATEGORY_EVENT,GENDER,COMPETITION,DATE,overall_rank
304,Neo Ee Nin,34.73,None,1.0,Discus (1.00kg),None,None,A,Throw,Female,NSG,2024,1
297,Emma Tan Yi Min,31.98,None,2.0,Discus (1.00kg),None,None,A,Throw,Female,NSG,2024,2
105,CHEN JING YING JANELLE,30.63,None,1.0,Discus (1.00kg),None,None,A,Throw,Female,NSG,2023,3
102,SHALEEN NUR ELYSA BINTE MUHAMMAD SHAHRIL,28.46,None,2.0,Discus (1.00kg),None,None,A,Throw,Female,NSG,2023,4
298,Thang Ying Yue,28.42,None,3.0,Discus (1.00kg),None,None,A,Throw,Female,NSG,2024,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,Ramesh Kaniyamuthan,11.22,None,8.0,Shot Put (5.00kg),None,None,A,Throw,Male,NSG,2024,15
215,Jadon Jusdi,10.83,None,9.0,Shot Put (5.00kg),None,None,A,Throw,Male,NSG,2024,16
214,Pranavv Ramkumar,10.38,None,10.0,Shot Put (5.00kg),None,None,A,Throw,Male,NSG,2024,17
217,Chen Xu Yan,9.97,None,11.0,Shot Put (5.00kg),None,None,A,Throw,Male,NSG,2024,18


In [2046]:
athletes_throwers=athletes_throwers[athletes_throwers['overall_rank']==1]

In [2047]:
athletes_throwers.to_csv('athletes_throwers.csv', sep=',', encoding='utf-8-sig', index=False)
